In [1]:
# # 如果没有安装 bokeh 可以将下面一行命令注释去掉之后并运行
# !pip install bokeh

# Part4 | 扩展：Bokeh与Pyechart 的介绍与使用（上）

## Bokeh 简介


![Image Name](https://raw.githubusercontent.com/bokeh/bokeh/branch-3.0/bokehjs/test/assets/images/logo.svg)

- 发音：/ˈbəʊ.keɪ/，释义：散景（也就是通常摄影圈所说「景深」）
- 不仅适用于 Python，也可以在 **JavaScript** 中使用
- Bokeh 的用途：
  1. 创建标准的可交互式图形
  2. 创建信息量更为丰富的 [Dashborad APP](https://docs.bokeh.org/en/latest/docs/gallery.html#server-app-examples)（也叫数据看板或数据仪表板）

## 一个示例

源：[Bokeh User Guide: Handling categorical data](https://docs.bokeh.org/en/latest/docs/user_guide/categorical.html#heatmaps)（代码略作修改）

In [2]:
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap

output_notebook()

Loading BokehJS ...

In [3]:
periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]
elements.head()

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,EA,standard state,bonding type,melting point,boiling point,density,metal,year discovered,group,period
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.20,37.0,NaN,120.0,...,-73.0,gas,diatomic,14.0,20.0,0.00009,nonmetal,1766,1,1
1,2,He,Helium,4.002602,#D9FFFF,1s2,NaN,32.0,NaN,140.0,...,0.0,gas,atomic,NaN,4.0,0.00000,noble gas,1868,18,1
2,3,Li,Lithium,6.941,#CC80FF,[He] 2s1,0.98,134.0,76 (+1),182.0,...,-60.0,solid,metallic,454.0,1615.0,0.54000,alkali metal,1817,1,2
3,4,Be,Beryllium,9.012182,#C2FF00,[He] 2s2,1.57,90.0,45 (+2),NaN,...,0.0,solid,metallic,1560.0,2743.0,1.85000,alkaline earth metal,1798,2,2
4,5,B,Boron,10.811,#FFB5B5,[He] 2s2 2p1,2.04,82.0,27 (+3),NaN,...,-27.0,solid,covalent network,2348.0,4273.0,2.46000,metalloid,1807,13,2


In [4]:
df = (
    elements.copy()
    .astype({"atomic mass": "str", "group": "str"})
    .assign(
        period=lambda d: [periods[x - 1] for x in d.period]
    )
    .loc[
        (elements.group != "-")
        & ~(elements.symbol.str.contains('L[ru]'))
    ]
)

df.head()

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,EA,standard state,bonding type,melting point,boiling point,density,metal,year discovered,group,period
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.20,37.0,NaN,120.0,...,-73.0,gas,diatomic,14.0,20.0,0.00009,nonmetal,1766,1,I
1,2,He,Helium,4.002602,#D9FFFF,1s2,NaN,32.0,NaN,140.0,...,0.0,gas,atomic,NaN,4.0,0.00000,noble gas,1868,18,I
2,3,Li,Lithium,6.941,#CC80FF,[He] 2s1,0.98,134.0,76 (+1),182.0,...,-60.0,solid,metallic,454.0,1615.0,0.54000,alkali metal,1817,1,II
3,4,Be,Beryllium,9.012182,#C2FF00,[He] 2s2,1.57,90.0,45 (+2),NaN,...,0.0,solid,metallic,1560.0,2743.0,1.85000,alkaline earth metal,1798,2,II
4,5,B,Boron,10.811,#FFB5B5,[He] 2s2 2p1,2.04,82.0,27 (+3),NaN,...,-27.0,solid,covalent network,2348.0,4273.0,2.46000,metalloid,1807,13,II


In [5]:
color_mapper = {
    "alkali metal": "#a6cee3",
    "alkaline earth metal": "#1f78b4",
    "metal": "#d93b43",
    "halogen": "#999d9a",
    "metalloid": "#e08d49",
    "noble gas": "#eaeaea",
    "nonmetal": "#f1d4Af",
    "transition metal": "#599d7A",
}

source = ColumnDataSource(df)

class Property(dict):
    def __init__(
        self,
        y,
        x=None,
        source=source,
        text_align="left",
        text_baseline="middle",
        text_font_size="16px",
        text_font_style="normal",
    ) -> None:
        super().__init__(
            y=y,
            x=x or dodge("group", -0.4, range=p.x_range),
            source=source,
            text_align=text_align,
            text_baseline=text_baseline,
            text_font_size={"value": text_font_size},
            text_font_style=text_font_style,
        )



In [6]:
p = figure(
    width=900,
    height=500,
    title="Periodic table (omitting LA and AC series)",
    x_range=groups,
    y_range=list(reversed(periods)),
    toolbar_location=None,
    tools="hover",
)

props = {
    "atomic number": Property(
        y=dodge("period", 0.3, range=p.y_range), text_font_size="11px"
    ),
    "symbol": Property(y="period", text_font_style="bold"),
    "atomic mass": Property(y=dodge("period", -0.2, range=p.y_range), text_font_size="7px"),
    "name": Property(y=dodge("period", -0.35, range=p.y_range), text_font_size="7px"),
}


In [7]:
show(p)

In [8]:

p.rect(
    x="group",
    y="period",
    width=0.95,
    height=0.95,
    source=source,
    fill_alpha=0.6,
    legend_field="metal",
    color=factor_cmap("metal", palette=list(color_mapper.values()), factors=list(color_mapper.keys())),
)

for name, prop in props.items():
    p.text(text=name, **prop)


p.text(
    x=["3", "3"],
    y=["VI", "VII"],
    text=["LA", "AC"],
    text_align="center",
    text_baseline="middle",
)

p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.legend.orientation = "horizontal"
p.legend.location = "top_center"

p.hover.tooltips = [
    ("Name", "@name"),
    ("Atomic number", "@{atomic number}"),
    ("Atomic mass", "@{atomic mass}"),
    ("Type", "@metal"),
    ("Electronic configuration", "@{electronic configuration}"),
]


show(p)

## 快速上手

Bokeh 的使用方式类似于 Matplotlib，在画布基础上调用相应的绘图方法，其使用公式为：

$$
\begin{aligned}
Bokeh &= \text{初始化 Figure 对象} 📐\\
      &+ \text{在 Figure 对象上调用相应绘图方法} 📊\\
      &+ \text{细节调整} 🛠\\
      &+ \text{渲染}
\end{aligned}
$$

In [9]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()   # <--- enable jupyter notebook mode here.

Loading BokehJS ...

In [10]:
import random
random.seed(233)

x = [random.randint(100, 20000) for _ in range(500)]
y = [v * random.uniform(0.1, 1) for v in x]
size = [random.randrange(1, 10) for _ in range(500)]

p = figure(title="Bokeh quickstart")


p.scatter(x, y, marker="+", size=size)
p.title.align = "center"
p.xaxis.axis_label = "X-Axis"
p.yaxis.axis_label = "Y-Axis"
show(p)

In [11]:
import random
random.seed(233)

import pandas as pd
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_mark, factor_cmap
x = [random.randint(-100, 20000) for _ in range(500)]
y = [v * random.uniform(0.1, 1) for v in x]
size = [random.randrange(1, 20) for _ in range(500)]
category = random.choices(list("ABC"), k=500)
df = pd.DataFrame(dict(
    x=x,
    y=y,
    size=size,
    category=category,
))


category = sorted(df["category"].unique())
data = ColumnDataSource(df)
df.head()

,x,y,size,category
0,5613,5510.421300,3,C
1,17108,2321.110956,15,B
2,7463,1705.812006,12,B
3,15763,4103.556755,3,B
4,17948,17689.827419,2,B


In [12]:

p = figure(title="Bokeh quickstart", tools="hover")


p.scatter(
    "x", 
    "y", 
    source=data,
    size="size", 
    fill_alpha=0.8,
    legend_group="category",
    marker=factor_mark(
        field_name="category", 
        markers=["triangle", "square", "hex"], 
        factors=category,
    ),
    color=factor_cmap(
        field_name="category",
        palette="Set2_3",
        factors=category,
    )
)


p.title.align = "center"
p.xaxis.axis_label = "X-Axis"
p.yaxis.axis_label = "Y-Axis"
p.legend.location = "top_left"
p.legend.title = "category"
p.hover.tooltips = [
    ("X", "@x"),
    ("Y", "@y"),
    ("Category", "@category"),
    ("Size", "@size"),
]
show(p)